In [1]:
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

#===========================================
# Definición del Dataset personalizado
#===========================================

In [2]:
class SargazoDataset(Dataset):
    def __init__(self, csv_path, img_dir, transform=None):
        self.data = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.transform = transform

        self.label_to_idx = None

        # Verificar si el CSV tiene más de una columna para crear el diccionario
        if self.data.shape[1] > 1:  # Verifica si hay al menos dos columnas
            self.label_to_idx = {label: idx for idx, label in enumerate(self.data.iloc[:, 1].unique())}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.data.iloc[idx, 0])  # Primer columna: nombre de imagen
        image = Image.open(img_name).convert('RGB')

        if self.transform:
            image = self.transform(image)

        # Si hay más de una columna (entrenamiento), obtenemos el label
        if self.label_to_idx is not None:
            label = self.data.iloc[idx, 1]  # Segunda columna: etiqueta
            label = torch.tensor(self.label_to_idx[label], dtype=torch.long)  # Convertimos a tensor
            return image, label
        else:
            return image

#===========================================
# Configuración del dispositivo (CPU/GPU)
#===========================================

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando dispositivo:", device)

Usando dispositivo: cuda


#===========================================
# Transformaciones para las imágenes
#===========================================

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

#===========================================
# Cargar los datasets de entrenamiento y test
#===========================================

In [9]:
train_csv = "clasifica-el-sargazo-24-b\\train_data.csv"
test_csv = "clasifica-el-sargazo-24-b\\test_data.csv"
img_dir = "clasifica-el-sargazo-24-b\\images_public"


train_dataset = SargazoDataset(csv_path=train_csv, img_dir=img_dir, transform=transform)
test_dataset = SargazoDataset(csv_path=test_csv, img_dir=img_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

N = 6  # Numero de clases [0, 1, 2, 3, 4, 5]

#===========================================
# Cargar el modelo GoogLeNet preentrenado
#===========================================

In [10]:
googlenet = models.googlenet(weights=True)

# Congelar capas base
# for param in googlenet.parameters():
#     param.requires_grad = False

# Reemplazar la última capa para nuestra tarea + Dropout
googlenet.fc = nn.Sequential(
    # nn.Dropout(p=0.2),
    nn.Linear(googlenet.fc.in_features, N)
)

googlenet = googlenet.to(device)

C:\Users\yleob\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


#===========================================
# Definir función de pérdida y optimizador
# Ir testeando diferentes funciones de pérdida y número de épocas
#===========================================

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(googlenet.parameters(), lr=0.005)

#===========================================
# Entrenamiento
#===========================================

In [12]:
# Definir la ruta del checkpoint
checkpoint_path = "checkpoint.pth"

# Función para guardar el checkpoint
def guardar_checkpoint(epoch, model, optimizer, loss, checkpoint_path):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }, checkpoint_path)
    print(f"Checkpoint guardado en la época {epoch + 1}")

# Función para cargar el checkpoint
def cargar_checkpoint(model, optimizer, checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    print(f"Checkpoint cargado. Última época entrenada: {epoch + 1}")
    return epoch, loss

# Entrenamiento
epochs_totales = 25  # Total de épocas que quieres entrenar
# epochs_totales = 15  # Solo para prueba
epoch_inicial = 0  # Valor por defecto si no se carga un checkpoint

# Intenta cargar un checkpoint si existe
try:
    epoch_inicial, loss = cargar_checkpoint(googlenet, optimizer, checkpoint_path)
    print(f"Continuando entrenamiento desde la época {epoch_inicial + 1}")
except FileNotFoundError:
    print("No se encontró un checkpoint. Entrenamiento iniciará desde el principio.")

# Comenzar el entrenamiento desde epoch_inicial
googlenet.train()  # modo entrenamiento
for epoch in range(epoch_inicial, epochs_totales):
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = googlenet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs_totales}, Loss: {avg_loss}")

    # Guardar checkpoint al final de cada época
    guardar_checkpoint(epoch, googlenet, optimizer, avg_loss, checkpoint_path)


Checkpoint cargado. Última época entrenada: 25
Continuando entrenamiento desde la época 25
Epoch 25/25, Loss: 0.7202747154545475
Checkpoint guardado en la época 25


#===========================================
# Evaluación en el conjunto de prueba
#===========================================

In [13]:
# Evaluar modelo y obtener predicciones
googlenet.eval()  # Cambiar a modo evaluación
googlenet = googlenet.to(device)

predictions = []  # Lista para almacenar las predicciones

with torch.no_grad():  # Desactiva cálculo de gradientes para inferencia
    for images in test_loader:  
        images = images.to(device)
        outputs = googlenet(images)
        _, predicted = torch.max(outputs.data, 1)  # Predicción como clase más probable
        predictions.extend(predicted.cpu().numpy())  # Guardar predicciones


In [14]:
# Diccionario para mapear etiquetas numéricas a etiquetas textuales
L = {1: "nada", 2: "bajo", 3: "moderado", 4: "abundante", 5: "excesivo"}

# Ajustar predicciones: cambiar 0 -> 1 y mapear al texto correspondiente
adjusted_predictions = [L.get(p if p > 0 else 1) for p in predictions]

# Cargar el CSV original
test_data = pd.read_csv(test_csv)

# Agregar columna de predicciones
test_data['Prediction'] = pd.Series(adjusted_predictions)

# Guardar el CSV actualizado
output_csv_path = "test_data_with_predictions.csv"
test_data.to_csv(output_csv_path, index=False)

print(f"Predicciones guardadas en: {output_csv_path}")

Predicciones guardadas en: test_data_with_predictions.csv
